In [1]:
import pandas as pd
import numpy as np
import os
import json
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler


import altair as alt
import hdbscan

import umap


In [2]:
datasets = [
    'boston', 
    'breastcancercoimbra',
    'breastcancerwisconsinprognostic',
    'covertype',
    'dermatology',
    'drybean',
    'echocardiogram',
    'ecoli',
    'extyaleb',
    'glassidentification',
    'heartdisease',
    'hepatitis',
    'housing', 'iris', 'mnist64', 'olive', 'weather', 'wine', 'world12d']

In [3]:
datasets_size = {
"boston": 70,
"breastcancercoimbra": 70,
"breastcancerwisconsinprognostic": 70,
"covertype": 65,
"dermatology": 70, #XX
"drybean": 65,
"echocardiogram": 70,
"ecoli": 70,
"extyaleb": 70,
"glassidentification": 70,
"heartdisease": 70,
"hepatitis": 65,
"housing": 65,
"iris": 70,
"mnist64": 70,
"olive": 70,  ##X
"weather": 70, 
"wine": 70, 
"world12d": 70
}

In [20]:
m = pd.read_json('./metric_all.json', orient='index')
s = pd.read_json('./scag_all.json', orient='index')
sm = pd.merge(s, m, left_index=True, right_index=True)


In [55]:
sm.sort_values(by=['Continuity'], ascending=False)

,outlying,convex,skinny,stringy,monotonic,skewed,clumpy,striated,sparse,RMSE,DTM_KL1,DTM_KL01,DTM_KL001,Sammon,Trustworthiness,Continuity,Spearman,Steadiness,Cohesiveness
drybean_49,0.142878,0.000330,0.794232,1.000000,0.029075,0.717090,0.877081,0.979487,0.012263,3.095568e+06,8.929251e-04,3.529643e-02,0.085011,0.999998,0.999799,0.999915,0.810426,0.961680,0.956526
drybean_45,0.150303,0.000270,0.506543,1.000000,0.038410,0.716813,0.881747,0.984536,0.011745,3.095568e+06,7.583988e-04,3.257272e-02,0.083454,0.999998,0.999794,0.999910,0.809558,0.955584,0.960876
wine_36,0.178726,0.769152,0.486579,0.725191,0.011810,0.629802,0.859283,0.450382,0.055713,1.122394e+01,1.019453e-08,1.852977e-07,0.000006,0.000070,0.999878,0.999909,0.989500,0.963634,0.981014
wine_40,0.185183,0.744772,0.419946,0.772059,0.015824,0.621107,0.874073,0.404412,0.052408,1.247391e+01,-4.815092e-08,1.003937e-07,0.000008,0.000085,0.999871,0.999895,0.987713,0.964219,0.978475
drybean_42,0.441596,0.110259,0.821145,0.783784,0.181930,0.762990,0.865582,0.479730,0.012572,9.296574e+03,-6.211121e-08,1.516892e-07,0.000005,0.000255,0.999856,0.999889,0.847162,0.985223,0.970032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
glassidentification_67,0.251083,0.277983,0.481177,0.733333,0.002823,0.834188,0.774517,0.366667,0.127295,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
glassidentification_66,0.314092,0.938955,0.137544,0.678571,0.055330,0.584164,0.692217,0.321429,0.110320,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
glassidentification_68,0.444655,0.772975,0.236763,0.666667,0.012925,0.682443,0.564506,0.407407,0.090681,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
glassidentification_69,0.314697,0.000000,1.000000,1.000000,1.000000,0.718113,0.892086,1.000000,0.057420,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000


In [46]:
scagnostics = ['outlying', 'convex', 'skinny', 'stringy', 'monotonic', 'skewed',
       'clumpy', 'striated', 'sparse']
# scagnostics = ['sparse']
scags = {}
num_per_scag = 100
num_per_row = 10
for scag in scagnostics:
       high = s.sort_values(by=[scag], ascending=False).index[:num_per_scag]
       charts_high = []
       for x in high:
              dataset = x.split('_')[0]
              numProj = x.split('_')[1]
              xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
              xy.columns = ['x', 'y']
              label = pd.read_json(f'./ld/{dataset}/label.json')
              label.columns = ['label']
              data = pd.merge(xy, label, left_index=True, right_index=True)
              with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
                     meta = json.load(f)
              c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_circle(opacity=.5, size=20).encode(
                     x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     color=alt.Color('label:N', legend=None)
              ).properties(width=150, height=150, title=alt.TitleParams(text=f'({round(s.loc[x][scag], 2)}) {dataset}-{numProj}, {meta["method"]}', limit=150, fontSize=10))
              charts_high.append(c)
       x = []
       for i in range(int(num_per_scag / num_per_row)):
              x.append(alt.hconcat(*charts_high[i*num_per_row:(i+1)*num_per_row], spacing=3))
       
       alt_high = alt.vconcat(*x).properties(title=alt.TitleParams(text=scag+" High 100", anchor='middle', color='darkblue', fontSize=20), spacing=3)
       
       low = s.sort_values(by=[scag], ascending=True).index[:num_per_scag]
       charts_low = []
       for x in low:
              dataset = x.split('_')[0]
              numProj = x.split('_')[1]
              xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
              xy.columns = ['x', 'y']
              label = pd.read_json(f'./ld/{dataset}/label.json')
              label.columns = ['label']
              data = pd.merge(xy, label, left_index=True, right_index=True)
              with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
                     meta = json.load(f)
              c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_circle(opacity=.5, size=20).encode(
                     x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     color=alt.Color('label:N', legend=None)
              ).properties(width=150, height=150, title=alt.TitleParams(text=f'({round(s.loc[x][scag], 2)}) {dataset}-{numProj}, {meta["method"]}', limit=150, fontSize=10))
              charts_low.append(c)
       x = []
       for i in range(int(num_per_scag / num_per_row)):
              x.append(alt.hconcat(*charts_low[i*num_per_row:(i+1)*num_per_row], spacing=3))
       
       alt_low = alt.vconcat(*x).properties(title=alt.TitleParams(text=scag+" low 100", anchor='middle', color='darkblue', fontSize=20), spacing=3)
       scags[scag] = (alt_high & alt_low)


In [ ]:
for s in scagnostics:
    print(f'<li><a href="./metric_viewer/scagpage/scag_{s}.html"><span style="color:#888">{s}</span></a></li>')

In [47]:
# scagnostics = ['outlying', 'convex', 'skinny', 'stringy', 'monotonic', 'skewed',
#        'clumpy', 'striated'. 'sparse']
idx = 7
for i in range(len(scagnostics)):
    scags[scagnostics[i]].save(f'./scag_{scagnostics[i]}.html')
# scags[scagnostics[0]]

In [99]:
charts = []
for x in a:
    dataset = x.split('_')[0]
    numProj = x.split('_')[1]
    xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
    xy.columns = ['x', 'y']
    label = pd.read_json(f'./ld/{dataset}/label.json')
    label.columns = ['label']
    data = pd.merge(xy, label, left_index=True, right_index=True)
    with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
        meta = json.load(f)
    c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_point().encode(
        x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
        y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
        color=alt.Color('label:N', legend=None)
    ).properties(width=150, height=150, title=f'{dataset}-{numProj}, {meta["method"]}')
    charts.append(c)
    

In [ ]:

x = []
for i in range(int(30 / 5)):
    x.append(alt.hconcat(*charts[i*5:(i+1)*5]))
    
alt.vconcat(*x).properties(title=alt.TitleParams(text=, anchor='middle'))

In [3]:
metric_type = [
    'DTM_KL1',
    'DTM_KL01',
    'DTM_KL001',
    'RMSE',
    'Sammon',
    'Spearman',
    'Trustworthiness',
    'Continuity',
    'Steadiness',
    'Cohesiveness'
    ]

In [4]:
normalized_data = {}
scaled_data = {}
raw = {}

for dataset in datasets:
    boston = pd.read_csv(f'./metric_raw/{dataset}_metrics.csv', index_col='num')
    raw[dataset] = boston

    method = boston["method"]
    boston.drop(['method'], axis=1, inplace=True)


    scaled  = StandardScaler().fit_transform(boston)

    normalized = preprocessing.normalize(boston, axis=0)
    np_normalized = np.array(normalized)
    
    boston["method"] = method
    kmeans_result = KMeans(n_clusters=10).fit(np_normalized).predict(np_normalized)
    hdbscan_result = hdbscan.HDBSCAN(min_cluster_size=3).fit_predict(np_normalized)
    normalized_data[dataset] = np_normalized
    scaled_data[dataset] = scaled
    boston["cluster_kmeans"] = kmeans_result
    boston["cluster_hdbscan"] = hdbscan_result
    cluster_kmeans = {}
    cluster_hdbscan = {}
    for idx, data in boston.groupby('cluster_kmeans'):
        cluster_kmeans[idx] = data.index.tolist()

    for idx, data in boston.groupby('cluster_hdbscan'):
        cluster_hdbscan[idx] = data.index.tolist()

    # with open(f'hdbscan/clustering_{dataset}.json', 'w') as f:
    #     json.dump(cluster_hdbscan, f)

    # with open(f'kmeans/clustering_{dataset}.json', 'w') as f:
    #     json.dump(cluster_kmeans, f)

    # boston.to_json(f'./metric/{dataset}_metrics.json', orient='index')
    

In [82]:
reducer = umap.UMAP(n_neighbors=3, min_dist=0.1, n_components=2)

for dataset in datasets[:1]:
    res_n = reducer.fit_transform(normalized_data[dataset])
    res_s = reducer.fit_transform(scaled_data[dataset])

    res_n = pd.DataFrame(res_n)
    res_s = pd.DataFrame(res_s)

    res_n.columns = ['x', 'y']
    res_s.columns = ['x', 'y']
    res_n.index.name = 'num'
    res_s.index.name = 'num'

    raw_d = raw[dataset]
    res_n = raw_d.merge(res_n, left_index=True, right_index=True)
    res_s = raw_d.merge(res_s, left_index=True, right_index=True)

    charts = alt.Chart(res_n).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
    ).properties(title=dataset+' normalized', width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
        ).properties(title=dataset+' standard', width=200, height=200)


for dataset in datasets[1:]:
    res_n = reducer.fit_transform(normalized_data[dataset])
    res_s = reducer.fit_transform(scaled_data[dataset])

    res_n = pd.DataFrame(res_n)
    res_s = pd.DataFrame(res_s)

    res_n.columns = ['x', 'y']
    res_s.columns = ['x', 'y']
    res_n.index.name = 'num'
    res_s.index.name = 'num'

    raw_d = raw[dataset]
    res_n = raw_d.merge(res_n, left_index=True, right_index=True)
    res_s = raw_d.merge(res_s, left_index=True, right_index=True)

    res_chart = alt.Chart(res_n).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
    ).properties(title=dataset+' normalized', width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
        ).properties(title=dataset+' standard', width=200, height=200)

    charts &= res_chart

In [ ]:
charts.title = 'UMAP(n_neighbors = 3, min_dist = 0.1, n_components=2)'
charts

In [5]:
from sklearn.manifold import TSNE

In [ ]:
chartss={}
for dataset in datasets[1:]:
    perplexity = 5

    tsne = TSNE(n_components=2, perplexity=perplexity)

    res_n = tsne.fit_transform(normalized_data[dataset])
    res_s = tsne.fit_transform(scaled_data[dataset])

    res_n = pd.DataFrame(res_n)
    res_s = pd.DataFrame(res_s)

    res_n.columns = ['x', 'y']
    res_s.columns = ['x', 'y']
    res_n.index.name = 'num'
    res_s.index.name = 'num'

    raw_d = raw[dataset]
    res_n = raw_d.merge(res_n, left_index=True, right_index=True)
    res_s = raw_d.merge(res_s, left_index=True, right_index=True)

    charts = alt.Chart(res_n).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
    ).properties(title='n /  perplexity : '+ str(perplexity), width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
        ).properties(title='s /perplexity : '+ str(perplexity), width=200, height=200)

    for perplexity in [10, 15, 20, 30, 40, 50]:
        tsne = TSNE(n_components=2, perplexity=perplexity)

        res_n = tsne.fit_transform(normalized_data[dataset])
        res_s = tsne.fit_transform(scaled_data[dataset])

        res_n = pd.DataFrame(res_n)
        res_s = pd.DataFrame(res_s)

        res_n.columns = ['x', 'y']
        res_s.columns = ['x', 'y']
        res_n.index.name = 'num'
        res_s.index.name = 'num'

        raw_d = raw[dataset]
        res_n = raw_d.merge(res_n, left_index=True, right_index=True)
        res_s = raw_d.merge(res_s, left_index=True, right_index=True)

        res_chart = alt.Chart(res_n).mark_circle(size=10).encode(
            x='x',
            y='y',
            color='method',
        ).properties(title='perplexity : '+ str(perplexity), width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
            x='x',
            y='y',
            color='method',
            ).properties(title='perplexity : '+ str(perplexity), width=200, height=200)

        charts &= res_chart

    charts.title = 't-SNE /' + dataset
    chartss[dataset] = charts




# for dataset in datasets[1:]:
#     res_n = tsne.fit_transform(normalized_data[dataset])
#     res_s = tsne.fit_transform(scaled_data[dataset])

#     res_n = pd.DataFrame(res_n)
#     res_s = pd.DataFrame(res_s)

#     res_n.columns = ['x', 'y']
#     res_s.columns = ['x', 'y']
#     res_n.index.name = 'num'
#     res_s.index.name = 'num'

#     raw_d = raw[dataset]
#     res_n = raw_d.merge(res_n, left_index=True, right_index=True)
#     res_s = raw_d.merge(res_s, left_index=True, right_index=True)

#     res_chart = alt.Chart(res_n).mark_circle(size=10).encode(
#         x='x',
#         y='y',
#         color='method',
#     ).properties(title=dataset+' normalized', width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
#         x='x',
#         y='y',
#         color='method',
#         ).properties(title=dataset+' standard', width=200, height=200)

#     charts &= res_chart